In [1]:
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
peliculas = pd.read_csv('../datos/peliculas.csv')
df_peli_artista = pd.read_csv('../datos/pelicula_artista.csv')
df_detalles_peli = pd.read_csv('../datos/detalles_peliculas.csv')
df_detalles_artistas = pd.read_csv('../datos/detalles_artistas.csv')

In [3]:
peliculas.head()

,Tipo,Titulo,Año,Mes,Id,generos
0,Movie,Las Travesuras de Super Chido,1990,10,tt0320656,Action
1,Movie,Velai Kidaichiruchu,1990,8,tt0318803,Action
2,Movie,Sor Batalla,1990,8,tt0320503,Action
3,Movie,Azaad Desh Ke Gulam,1990,4,tt0330019,Action
4,Movie,Suo ming fei dao,1991,11,tt0348147,Action


In [4]:
peliculas.info() # no tengo nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453 entries, 0 to 1452
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Tipo     1453 non-null   object
 1   Titulo   1453 non-null   object
 2   Año      1453 non-null   int64 
 3   Mes      1453 non-null   int64 
 4   Id       1453 non-null   object
 5   generos  1453 non-null   object
dtypes: int64(2), object(4)
memory usage: 68.2+ KB


In [5]:
peliculas['Tipo'].unique() # tengo 4 tipo de elementos en mis datos

array(['Movie', 'Short', 'TV Movie', 'TV Short'], dtype=object)

In [6]:
peliculas.nunique()

Tipo          4
Titulo     1444
Año          35
Mes          13
Id         1453
generos       8
dtype: int64

## Detalles Películas

In [7]:
df_detalles_peli.head()

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
0,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,tt0320656
1,Velai Kidaichiruchu,5.2,P. Vasu,SathyarajGautamiAnandraj,"Baasha, a gangster, controls a whole village w...",Desconocido,tt0318803
2,Sor Batalla,6.6,Jesús Fragoso Montoya,Fernando Galiana,As punishment for participating in a contest T...,1h 34min,tt0320503
3,Azaad Desh Ke Gulam,5.3,S.A. Chandrashekhar,RekhaRishi KapoorJackie Shroff,"Bharti, a law student from a rich family, live...",2h 33min,tt0330019
4,Suo ming fei dao,NaN,Desconocido,Desconocido,Desconocido,1h 24min,tt0348147


In [8]:
# df_detalles_peli['durac_imbd'] = df_detalles_peli['durac_imbd'].str.replace('h','')
# df_detalles_peli['durac_imbd'] = df_detalles_peli['durac_imbd'].str.replace('min','')
# df_detalles_peli['durac_imbd'] = df_detalles_peli['durac_imbd'].str.strip()

In [9]:
df_detalles_peli.head()

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
0,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,tt0320656
1,Velai Kidaichiruchu,5.2,P. Vasu,SathyarajGautamiAnandraj,"Baasha, a gangster, controls a whole village w...",Desconocido,tt0318803
2,Sor Batalla,6.6,Jesús Fragoso Montoya,Fernando Galiana,As punishment for participating in a contest T...,1h 34min,tt0320503
3,Azaad Desh Ke Gulam,5.3,S.A. Chandrashekhar,RekhaRishi KapoorJackie Shroff,"Bharti, a law student from a rich family, live...",2h 33min,tt0330019
4,Suo ming fei dao,NaN,Desconocido,Desconocido,Desconocido,1h 24min,tt0348147


In [10]:
df_detalles_peli['durac_imbd'][0][0]

'1'

In [11]:
df_detalles_peli.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453 entries, 0 to 1452
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   titulo       1453 non-null   object 
 1   cali_imdb    961 non-null    float64
 2   direc_imbd   1453 non-null   object 
 3   guion_imbd   1453 non-null   object 
 4   argum_imbd   1453 non-null   object 
 5   durac_imbd   1453 non-null   object 
 6   id_pelicula  1453 non-null   object 
dtypes: float64(1), object(6)
memory usage: 79.6+ KB


In [12]:
df_detalles_peli['cali_imdb'].fillna(0, inplace=True)

In [13]:
df_detalles_peli

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
0,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,tt0320656
1,Velai Kidaichiruchu,5.2,P. Vasu,SathyarajGautamiAnandraj,"Baasha, a gangster, controls a whole village w...",Desconocido,tt0318803
2,Sor Batalla,6.6,Jesús Fragoso Montoya,Fernando Galiana,As punishment for participating in a contest T...,1h 34min,tt0320503
3,Azaad Desh Ke Gulam,5.3,S.A. Chandrashekhar,RekhaRishi KapoorJackie Shroff,"Bharti, a law student from a rich family, live...",2h 33min,tt0330019
4,Suo ming fei dao,0.0,Desconocido,Desconocido,Desconocido,1h 24min,tt0348147
...,...,...,...,...,...,...,...
1448,Der Wien-Krimi: Blind ermittelt - Tod an der D...,6.2,Anne Zohra Berrached,Mike Majzen,Mysteriöser Doppelmord an der Donau. Eine Frau...,1h 29min,tt27251096
1449,Marne Ke Dar Se Mere Dil,0.0,Priyasha Bhardwaj,Desconocido,"One fateful night , Samadhan reluctantly goes ...",Desconocido,tt27250288
1450,Red Wine,0.0,Desconocido,Desconocido,Desconocido,Desconocido,tt27250168
1451,Magna Lacerta,0.0,Ruby Fuller,Ruby FullerKevin Lucero LessJacob Schindler,Two students produce a documentary about the d...,Desconocido,tt27262238


## Artistas por película

In [14]:
df_peli_artista.head()

,Movie ID,Actriz_Actor
0,tt0110660,Tony Ka Fai Leung
1,tt0110363,Luke Edwards
2,tt0110397,Jennifer Barker
3,tt0111045,Miklós Déri
4,tt0111653,Joe Bays


In [15]:
df_peli_artista[df_peli_artista['Actriz_Actor'] == 'A.C. Peterson']

,Movie ID,Actriz_Actor
6667,tt1151400,A.C. Peterson


In [16]:
df_peli_artista.info() # no hay nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14529 entries, 0 to 14528
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Movie ID      14529 non-null  object
 1   Actriz_Actor  14529 non-null  object
dtypes: object(2)
memory usage: 227.1+ KB


## Detalles de artistas

In [17]:
df_detalles_artistas.head()

,actr,año_nac,conocido_por,que_hace,premios
0,A. Gunaseelan,9999,'Mandela',"Casting Department, Actress, Casting Director",1 nomination
1,A. Michelle Harleston,9999,"'Murder by Numbers', 'The Mark of a Killer', '...",Actor,no tiene premios
2,A.C. Peterson,9999,"'Narc', 'Shanghai Noon', 'Shooter', 'The Last ...",Actor,no tiene premios
3,A.V.S. Subramanyam,1957,"'Roommates', 'Uncle', 'Ori Nee Prema Bangaramk...","Actor, Director, Writer",no tiene premios
4,Øyvind Øvrebø,9999,"'Tingen, Edderkoppen', 'Demenskoret', 'Kristia...","Actress, Additional Crew",no tiene premios


In [18]:
df_detalles_artistas.info() # no tiene nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9907 entries, 0 to 9906
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   actr          9907 non-null   object
 1   año_nac       9907 non-null   int64 
 2   conocido_por  9907 non-null   object
 3   que_hace      9907 non-null   object
 4   premios       9907 non-null   object
dtypes: int64(1), object(4)
memory usage: 387.1+ KB


In [19]:
df_detalles_artistas.describe() # considerando que los datos son mayormente erróneos, me cargaré esa columna

,año_nac
count,9907.000000
mean,6685.205309
std,3955.479167
min,1860.000000
25%,1971.000000
50%,9999.000000
75%,9999.000000
max,9999.000000


In [20]:
df_detalles_artistas.drop(columns='año_nac', inplace=True)

In [21]:
df_detalles_artistas.head()

,actr,conocido_por,que_hace,premios
0,A. Gunaseelan,'Mandela',"Casting Department, Actress, Casting Director",1 nomination
1,A. Michelle Harleston,"'Murder by Numbers', 'The Mark of a Killer', '...",Actor,no tiene premios
2,A.C. Peterson,"'Narc', 'Shanghai Noon', 'Shooter', 'The Last ...",Actor,no tiene premios
3,A.V.S. Subramanyam,"'Roommates', 'Uncle', 'Ori Nee Prema Bangaramk...","Actor, Director, Writer",no tiene premios
4,Øyvind Øvrebø,"'Tingen, Edderkoppen', 'Demenskoret', 'Kristia...","Actress, Additional Crew",no tiene premios


## Trabajo de tablas para inserción

In [22]:
df_pelis_full = df_detalles_peli.merge(peliculas, how='inner', left_on= 'id_pelicula', right_on= 'Id')

In [23]:
df_pelis_full.head()

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula,Tipo,Titulo,Año,Mes,Id,generos
0,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,tt0320656,Movie,Las Travesuras de Super Chido,1990,10,tt0320656,Action
1,Velai Kidaichiruchu,5.2,P. Vasu,SathyarajGautamiAnandraj,"Baasha, a gangster, controls a whole village w...",Desconocido,tt0318803,Movie,Velai Kidaichiruchu,1990,8,tt0318803,Action
2,Sor Batalla,6.6,Jesús Fragoso Montoya,Fernando Galiana,As punishment for participating in a contest T...,1h 34min,tt0320503,Movie,Sor Batalla,1990,8,tt0320503,Action
3,Azaad Desh Ke Gulam,5.3,S.A. Chandrashekhar,RekhaRishi KapoorJackie Shroff,"Bharti, a law student from a rich family, live...",2h 33min,tt0330019,Movie,Azaad Desh Ke Gulam,1990,4,tt0330019,Action
4,Suo ming fei dao,0.0,Desconocido,Desconocido,Desconocido,1h 24min,tt0348147,Movie,Suo ming fei dao,1991,11,tt0348147,Action


In [24]:
peliculas.drop(columns=['Año', 'Mes', 'generos'], inplace=True)

In [25]:
peliculas.head() #será una tabla

,Tipo,Titulo,Id
0,Movie,Las Travesuras de Super Chido,tt0320656
1,Movie,Velai Kidaichiruchu,tt0318803
2,Movie,Sor Batalla,tt0320503
3,Movie,Azaad Desh Ke Gulam,tt0330019
4,Movie,Suo ming fei dao,tt0348147


In [26]:
df_pelis_full.drop(columns='id_pelicula', inplace=True) #tabla principal

In [27]:
df_pelis_full.head()

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,Tipo,Titulo,Año,Mes,Id,generos
0,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,Movie,Las Travesuras de Super Chido,1990,10,tt0320656,Action
1,Velai Kidaichiruchu,5.2,P. Vasu,SathyarajGautamiAnandraj,"Baasha, a gangster, controls a whole village w...",Desconocido,Movie,Velai Kidaichiruchu,1990,8,tt0318803,Action
2,Sor Batalla,6.6,Jesús Fragoso Montoya,Fernando Galiana,As punishment for participating in a contest T...,1h 34min,Movie,Sor Batalla,1990,8,tt0320503,Action
3,Azaad Desh Ke Gulam,5.3,S.A. Chandrashekhar,RekhaRishi KapoorJackie Shroff,"Bharti, a law student from a rich family, live...",2h 33min,Movie,Azaad Desh Ke Gulam,1990,4,tt0330019,Action
4,Suo ming fei dao,0.0,Desconocido,Desconocido,Desconocido,1h 24min,Movie,Suo ming fei dao,1991,11,tt0348147,Action


In [28]:
df_peli_artista.nunique()

Movie ID        1452
Actriz_Actor    9909
dtype: int64

In [29]:
df_central = df_pelis_full.merge(df_peli_artista, how='left', left_on= 'Id', right_on= 'Movie ID')

In [30]:
# df_pelis_full['prueba'] = df_peli_artista[df_peli_artista['Movie ID'] == 'tt0293357']['Actriz_Actor'].tolist()

In [31]:
df_central.head()

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,Tipo,Titulo,Año,Mes,Id,generos,Movie ID,Actriz_Actor
0,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,Movie,Las Travesuras de Super Chido,1990,10,tt0320656,Action,tt0320656,Luis de Alba
1,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,Movie,Las Travesuras de Super Chido,1990,10,tt0320656,Action,tt0320656,Óscar Fentanes
2,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,Movie,Las Travesuras de Super Chido,1990,10,tt0320656,Action,tt0320656,Gabriela Goldsmith
3,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,Movie,Las Travesuras de Super Chido,1990,10,tt0320656,Action,tt0320656,Charly Valentino
4,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,Movie,Las Travesuras de Super Chido,1990,10,tt0320656,Action,tt0320656,Jorge Arvizu


In [32]:
df_central.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14530 entries, 0 to 14529
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   titulo        14530 non-null  object 
 1   cali_imdb     14530 non-null  float64
 2   direc_imbd    14530 non-null  object 
 3   guion_imbd    14530 non-null  object 
 4   argum_imbd    14530 non-null  object 
 5   durac_imbd    14530 non-null  object 
 6   Tipo          14530 non-null  object 
 7   Titulo        14530 non-null  object 
 8   Año           14530 non-null  int64  
 9   Mes           14530 non-null  int64  
 10  Id            14530 non-null  object 
 11  generos       14530 non-null  object 
 12  Movie ID      14529 non-null  object 
 13  Actriz_Actor  14529 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 1.6+ MB


In [33]:
df_central.dropna(inplace=True)

In [34]:
df_central.reset_index(drop=True, inplace=True)

In [35]:
df_central.drop(columns=['Movie ID', 'Tipo', 'Titulo'], inplace=True)

In [36]:
df_central.head()

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,Año,Mes,Id,generos,Actriz_Actor
0,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,1990,10,tt0320656,Action,Luis de Alba
1,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,1990,10,tt0320656,Action,Óscar Fentanes
2,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,1990,10,tt0320656,Action,Gabriela Goldsmith
3,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,1990,10,tt0320656,Action,Charly Valentino
4,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,Desconocido,1h 35min,1990,10,tt0320656,Action,Jorge Arvizu


# Llamada a BBDD

In [37]:
try:
    conexion = psycopg2.connect(
    database = 'Prueba',
    user = 'postgres',
    password = 'admin',
    host = 'localhost',
    port = '5432')
except OperationalError as e: #el alias es e
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print('la contraseña es erronea')
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print('error de conexion')
    else:
        print(f'el error es el siguiente {e}')

## Creación de tablas

In [38]:
peliculas.head()

,Tipo,Titulo,Id
0,Movie,Las Travesuras de Super Chido,tt0320656
1,Movie,Velai Kidaichiruchu,tt0318803
2,Movie,Sor Batalla,tt0320503
3,Movie,Azaad Desh Ke Gulam,tt0330019
4,Movie,Suo ming fei dao,tt0348147


In [39]:
peliculas = peliculas.iloc[:, [2, 0, 1]]

In [40]:
peliculas.head()

,Id,Tipo,Titulo
0,tt0320656,Movie,Las Travesuras de Super Chido
1,tt0318803,Movie,Velai Kidaichiruchu
2,tt0320503,Movie,Sor Batalla
3,tt0330019,Movie,Azaad Desh Ke Gulam
4,tt0348147,Movie,Suo ming fei dao


In [41]:
cursor = conexion.cursor()
tabla_audiovisual = """
                   CREATE TABLE IF NOT EXISTS audiovisual (
                   id VARCHAR(50) PRIMARY KEY,
                   tipo VARCHAR(20),
                   titulo VARCHAR (100)
                   );
                   """
cursor.execute(tabla_audiovisual)
conexion.commit()

In [42]:
cursor = conexion.cursor()
tabla_actoral = """
                   CREATE TABLE IF NOT EXISTS actores (
                   nombre VARCHAR(100) PRIMARY KEY,
                   conocido_por VARCHAR(500),
                   que_hace VARCHAR(100),
                   premios VARCHAR(500)
                   );
                   """
cursor.execute(tabla_actoral)
conexion.commit()

In [43]:
cursor = conexion.cursor()
tabla_contenido = """
                   CREATE TABLE IF NOT EXISTS central (
                   titulo VARCHAR(100),
                   cal_imdb float,
                   direc_imbd VARCHAR(200),
                   guion_imdb VARCHAR(100),
                   argum_imbd VARCHAR(500),
                   durac_imbd VARCHAR(50),
                   Anio int,
                   mes int,
                   id_movie VARCHAR(50),
                   generos VARCHAR(50),
                   actoral VARCHAR(50),
                    CONSTRAINT fk_audiovisual FOREIGN KEY (id_movie)
                        REFERENCES audiovisual(id),
                    CONSTRAINT fk_actores FOREIGN KEY (actoral)
                        REFERENCES actores(nombre)
                   );
                   """
cursor.execute(tabla_contenido)
conexion.commit()

### Inserción

In [44]:
lista_tuplas = []
for fila in peliculas.values:
    lista_tuplas.append(tuple(fila))

query_carga = 'INSERT INTO audiovisual (id, tipo, titulo) VALUES (%s, %s, %s)'

for valores in lista_tuplas:
    cursor.execute(query_carga, valores)
conexion.commit()

In [45]:
lista_tuplas_2 = []
for fila in df_detalles_artistas.values:
    lista_tuplas_2.append(tuple(fila))

query_carga2 = 'INSERT INTO actores (nombre, conocido_por, que_hace, premios) VALUES (%s, %s, %s, %s)'

for valores in lista_tuplas_2:
    cursor.execute(query_carga2, valores)
conexion.commit()

In [47]:
lista_tuplas_3 = []
for fila in df_central.values:
    lista_tuplas_3.append(tuple(fila))

query_carga3 = 'INSERT INTO central (titulo, cal_imdb, direc_imbd, guion_imdb,argum_imbd, durac_imbd, Anio, mes, id_movie, generos, actoral) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

for valores in lista_tuplas_3:
    cursor.execute(query_carga3, valores)
conexion.commit()

ForeignKeyViolation: inserción o actualización en la tabla «central» viola la llave foránea «fk_actores»
DETAIL:  La llave (actoral)=(James McDaniel) no está presente en la tabla «actores».


In [87]:
df_central.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14529 entries, 0 to 14528
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   titulo        14529 non-null  object 
 1   cali_imdb     14529 non-null  float64
 2   direc_imbd    14529 non-null  object 
 3   guion_imbd    14529 non-null  object 
 4   argum_imbd    14529 non-null  object 
 5   durac_imbd    14529 non-null  object 
 6   Año           14529 non-null  int64  
 7   Mes           14529 non-null  int64  
 8   Id            14529 non-null  object 
 9   generos       14529 non-null  object 
 10  Actriz_Actor  14529 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 1.2+ MB


In [47]:
df_central.values[0]

array(['Las Travesuras de Super Chido', 7.7, 'Alejandro Todd',
       'Óscar FentanesJuan Garrido', 'Desconocido', '1h 35min', 1990, 10,
       'tt0320656', 'Action', 'Luis de Alba'], dtype=object)